At first, we load **libraries with functions needed for analysis** (cobra), dataframe and graph generation, as well as requests for access to open APIs.

In [1]:
import cobra
cobra.Configuration().solver='cplex'
import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import mygene
import requests, sys
from Bio import Entrez
from chembl_webresource_client.new_client import new_client

GUROBI is available but could not load with error:
  Traceback (most recent call last):
    File "/usr/local/lib/python3.6/site-packages/optlang/gurobi_interface.py", line 40, in <module>
      raise RuntimeError()
  RuntimeError
  
  During handling of the above exception, another exception occurred:
  
  Traceback (most recent call last):
    File "/usr/local/lib/python3.6/site-packages/optlang/__init__.py", line 49, in <module>
      from optlang import gurobi_interface
    File "/usr/local/lib/python3.6/site-packages/optlang/gurobi_interface.py", line 43, in <module>
      "This version of optlang requires a Gurobi version of 9.5 or above.")
  RuntimeError: This version of optlang requires a Gurobi version of 9.5 or above.


**Loading reaction and model data and information**
- namesBase, namesMM, namesTissues
- Remove the last 7 letters: name = name[:len(name)-7].
- Names is a list of all 49 names
- Two dictionaries: indexModel[model], modelName[index], modelName[index].
- modelBase : generic model Human-GEM
- myGroup[r.id] : name of the subsystem it belongs to
- reactionsIncluded[model] : ids of the reactions of the model

In [2]:
nombresBase=os.listdir("/home/alumno15/MCS/base/")
nombresTejidos=os.listdir("/home/alumno15/MCS/tejidos/")
nombresMM=os.listdir("/home/alumno15/MCS/MM/")

In [3]:
nombres=[]
for nombre in nombresBase:
    nombres.append(nombre[:len(nombre)-7])
for nombre in nombresTejidos:
    nombres.append(nombre[:len(nombre)-7])
for nombre in nombresMM:
    nombres.append(nombre[:len(nombre)-7])

In [4]:
nombreModelo={}
indiceModelo={}
for i,modelo in enumerate(nombres):
    nombreModelo[i]=modelo
    indiceModelo[modelo]=i

In [5]:
with open("/home/alumno15/modelos/base/Human-GEM.dat","rb") as f:
    modeloBase=pickle.load(f)

In [6]:
miGrupo={}
for group in modeloBase.groups:
    for r in group.members:
        miGrupo[r.id]=group.name

In [7]:
'''
reaccionesIncluidas={}
for nombre in nombresMM:
    nombre=nombre[:len(nombre)-7]
    with open("/home/alumno15/modelos/MM/"+nombre+".dat","rb") as f:
        modelo=pickle.load(f)
    reaccionesIncluidas[nombre]=set([r.id for r in modelo.reactions])
for nombre in nombresTejidos:
    nombre=nombre[:len(nombre)-7]
    with open("/home/alumno15/modelos/tejidos/"+nombre+".dat","rb") as f:
        modelo=pickle.load(f)
    reaccionesIncluidas[nombre]=set([r.id for r in modelo.reactions])

with open("reaccionesIncluidas.dat","wb") as f:
    pickle.dump(reaccionesIncluidas,f)
'''

with open("reaccionesIncluidas.dat","rb") as f:
    reaccionesIncluidas = pickle.load(f)

**Tools to analyse MCS**
- MCSByTask [model] [task]: dictionary storing MCS of all models classified by metabolic task
- MCSAllTasks [model]: dictionary storing all MCSs of all models without distinguishing by metabolic task
- We load a single line, in this case it will be **MMIS**

In [8]:
MCSByTask={}
MCSAllTasks={}

for nombre in nombresBase:
    nombre=nombre[:len(nombre)-7]
    with open("/home/alumno15/MCS/base/"+nombre+"MCS.dat","rb") as f:
        MCS=pickle.load(f)
    todos=set()
    for task in range(57):
        todos.update(MCS[task][0])
    ByTask=[set()]
    for task in range(57):
        ByTask.append(MCS[task][0])
    MCSByTask[nombre]=ByTask
    MCSAllTasks[nombre]=todos
    
for nombre in nombresTejidos:
    nombre=nombre[:len(nombre)-7]
    with open("/home/alumno15/MCS/tejidos/"+nombre+"MCS.dat","rb") as f:
        MCS=pickle.load(f)
    todos=set()
    for task in range(57):
        todos.update(MCS[task][0])
    ByTask=[set()]
    for task in range(57):
        ByTask.append(MCS[task][0])
    MCSByTask[nombre]=ByTask
    MCSAllTasks[nombre]=todos
    
#Specify line by changing model ID
nombre="ACH_000763"
with open("/home/alumno15/MCS/MM/"+nombre+"MCS.dat","rb") as f:
    MCS=pickle.load(f)
todos=set()
for task in range(57):
    todos.update(MCS[task][0])
ByTask=[set()]
for task in range(57):
    ByTask.append(MCS[task][0])
MCSByTask[nombre]=ByTask
MCSAllTasks[nombre]=todos

**Toxicity studies (MCSs that affect healthy tissues)**
- modelsAffected[r.id] : list of tissues affected by some MCSs
- numeroModelosAfectados[r.id] : list of the ‘number’ of tissues affected by some MCSs
- Level1,..., Level5: toxicity levels
- generic: MCS of the base model
- someModel: MCSs which block some tissue
- specific: MCSs which block some tissue and are not generic

In [9]:
genericas=MCSAllTasks[nombreModelo[0]]
print("Number of MCSs in the generic model:",len(genericas))

Number of MCSs in the generic model: 441


In [10]:
algunModelo=set()
for i in range(1,31):
    especificasTask=MCSAllTasks[nombreModelo[i]]
    algunModelo.update(especificasTask)
print("Number of MCSs in some healthy tissue model:",len(algunModelo))

Number of MCSs in some healthy tissue model: 912


In [11]:
especificas=algunModelo.difference(genericas)
print("Number of MCSs specific to healthy tissue models:",len(especificas))

Number of MCSs specific to healthy tissue models: 471


In [12]:
numeroModelosAfectados={}
modelosAfectados={}
for r in especificas:
    afectados=set()
    for i in range(1,31):
        if r in MCSAllTasks[nombreModelo[i]]:
            afectados.add(nombreModelo[i])
    numeroModelosAfectados[r]=len(afectados)
    modelosAfectados[r]=afectados

**Target studies (MCSs that affect KMM1 line in particular)**
- targetsModeloMM: MCSs which block some line
- especificasLineaMM: MCSs which block some line and are not generic

In [15]:
targetsModeloMM=set()
especificasTask=MCSAllTasks["ACH_000763"] #Specify line by changing model ID
targetsModeloMM.update(especificasTask)
len(targetsModeloMM)

680

In [16]:
especificasLineaMM=targetsModeloMM.difference(genericas)
len(especificasLineaMM)

239

**Classification by range of toxicities**

- We are grouping the MCS found for the line considered based on the number of healthy tissues they affect. 
- If they do not affect any of them, said MCS will not have been included in the "numeroModelosAfectados" dictionary and therefore we must show it and consider it, since it will not present toxicities in healthy tissue.

In [17]:
#Initialize counters and lists for each category
cont = 0
menos_de_5 = []
entre_5_y_9 = []
entre_10_y_18 = []
entre_20_y_29 = []

#Iterate through the list and categorize
for r in especificasLineaMM:
    if r in numeroModelosAfectados:
        count = numeroModelosAfectados[r]
        if count < 5:
            menos_de_5.append((r, count))
        elif 5 <= count < 10:
            entre_5_y_9.append((r, count))
        elif 10 <= count < 19:
            entre_10_y_18.append((r, count))
        elif 20 <= count < 30:
            entre_20_y_29.append((r, count))
        else:
            cont += 1
    else:
        print(f"Warning: Key '{r}' not found in numeroModelosAfectados")

#Create a DataFrame for each category
df_menos_de_5 = pd.DataFrame(menos_de_5, columns=['ID Reaction', 'Toxicities'])
df_entre_5_y_9 = pd.DataFrame(entre_5_y_9, columns=['ID Reaction', 'Toxicities'])
df_entre_10_y_18 = pd.DataFrame(entre_10_y_18, columns=['ID Reaction', 'Toxicities'])
df_entre_20_y_29 = pd.DataFrame(entre_20_y_29, columns=['ID Reaction', 'Toxicities'])

#Print the results in blocks
if not df_menos_de_5.empty:
    print("MCS que tienen menos de 5 toxicidades")
    print(df_menos_de_5)

if not df_entre_5_y_9.empty:
    print("\nMCS que tienen entre 5-9 toxicidades")
    print(df_entre_5_y_9)

if not df_entre_10_y_18.empty:
    print("\nMCS que tienen entre 10-18 toxicidades")
    print(df_entre_10_y_18)

if not df_entre_20_y_29.empty:
    print("\nMCS que tienen entre 20-29 toxicidades")
    print(df_entre_20_y_29)

#Print the count of toxicities almost generic
print(f"\nMas {cont} toxicidades casi genericas")

MCS que tienen menos de 5 toxicidades
  ID Reaction  Toxicities
0    MAR06644           1
1    MAR06645           4
2    MAR02299           3
3    MAR06466           4
4    MAR08702           1
5    MAR08710           4
6    MAR06464           4
7    MAR06660           1

MCS que tienen entre 5-9 toxicidades
   ID Reaction  Toxicities
0     MAR04917           8
1     MAR03386           8
2     MAR00638           8
3     MAR01648           8
4     MAR01645           8
5     MAR04336           7
6     MAR02594           9
7     MAR00636           8
8     MAR00651           7
9     MAR03037           9
10    MAR00589           8
11    MAR06476           5
12    MAR06505           7
13    MAR00597           8
14    MAR05029           7
15    MAR04898           6
16    MAR00067           8
17    MAR06478           5
18    MAR00625           8

MCS que tienen entre 10-18 toxicidades
   ID Reaction  Toxicities
0     MAR00526          11
1     MAR00517          11
2     MAR00521          11
3 

**Comparison with candidate MCS for various MM lines**

- The next step is to consider the MCS obtained as promising in the analysis for several MM lines (buenasTodas: they kill a minimum of 10 lines and present a maximum of 5 different toxicities) and see if they are also promising for this specific line.
- Explore which are promising MCS unique to this line (set(buenasLinea).difference(buenasTodas)), the metabolites associated with the reaction and the subsystem to which they belong.
- Furthermore, we will investigate the identity of the toxicities associated with some of them.

In [18]:
buenasTodas=set(["MAR04623", "MAR02598", "MAR04473", "MAR06660", "MAR06644", "MAR08702", "MAR06386"])

In [19]:
noToxicities=[r for r in especificasLineaMM if r not in numeroModelosAfectados]
noToxicities

['MAR04473',
 'MAR20111',
 'MAR04922',
 'MAR06662',
 'MAR08009',
 'MAR07702',
 'MAR06386',
 'MAR02598',
 'MAR04623',
 'MAR07708']

In [20]:
buenasLinea=set([caso[0] for caso in menos_de_5])
buenasLinea.update(noToxicities)
for r in set(buenasLinea).difference(buenasTodas):
    if r not in numeroModelosAfectados:
        afecta=0
    else:
        afecta=numeroModelosAfectados[r]
    print(r,"Toxicidades",afecta,miGrupo[r])

MAR07708 Toxicidades 0 Transport reactions
MAR06464 Toxicidades 4 Vitamin E metabolism
MAR20111 Toxicidades 0 Miscellaneous
MAR04922 Toxicidades 0 Transport reactions
MAR08009 Toxicidades 0 Transport reactions
MAR07702 Toxicidades 0 Glycine, serine and threonine metabolism
MAR06645 Toxicidades 4 Retinol metabolism
MAR08710 Toxicidades 4 Retinol metabolism
MAR06662 Toxicidades 0 Retinol metabolism
MAR02299 Toxicidades 3 Transport reactions
MAR06466 Toxicidades 4 Vitamin E metabolism


In [21]:
for r in especificasLineaMM:
    if r not in numeroModelosAfectados:
        print(r, miGrupo[r])

MAR04473 Pentose phosphate pathway
MAR20111 Miscellaneous
MAR04922 Transport reactions
MAR06662 Retinol metabolism
MAR08009 Transport reactions
MAR07702 Glycine, serine and threonine metabolism
MAR06386 Transport reactions
MAR02598 Carnitine shuttle (mitochondrial)
MAR04623 Pentose phosphate pathway
MAR07708 Transport reactions


**MCS analysis with less than 5 toxicities**

In [22]:
not_found = []

for r in especificasLineaMM:
    if r in numeroModelosAfectados:
        if numeroModelosAfectados[r] < 5:
            print(r, "||", miGrupo[r])
            print(modelosAfectados[r])
            print()
    else:
        not_found.append(r)

if not_found:
    print("Reacciones no encontradas en numeroModelosAfectados:")
    for r in not_found:
        print(r)

MAR06644 || Retinol metabolism
{'blood'}

MAR06645 || Retinol metabolism
{'spleen', 'thyroid', 'esophagus', 'brain'}

MAR02299 || Transport reactions
{'blood_vessel', 'pancreas', 'heart'}

MAR06466 || Vitamin E metabolism
{'esophagus', 'colon', 'heart', 'ovary'}

MAR08702 || Retinol metabolism
{'blood'}

MAR08710 || Retinol metabolism
{'spleen', 'thyroid', 'esophagus', 'brain'}

MAR06464 || Vitamin E metabolism
{'esophagus', 'colon', 'heart', 'ovary'}

MAR06660 || Retinol metabolism
{'blood'}

Reacciones no encontradas en numeroModelosAfectados:
MAR04473
MAR20111
MAR04922
MAR06662
MAR08009
MAR07702
MAR06386
MAR02598
MAR04623
MAR07708


**MCS analysis with associated toxicities ranging from 5 to 10**

In [23]:
not_found2 = []

for r in especificasLineaMM:
    if r in numeroModelosAfectados:
        if 5 <= numeroModelosAfectados[r] < 10:
            print(r, "||", miGrupo[r])
            print(modelosAfectados[r])
            print()
    else:
        not_found2.append(r)

if not_found2:
    print("Reacciones no encontradas en numeroModelosAfectados:")
    for r in not_found2:
        print(r)

MAR04917 || Transport reactions
{'muscle', 'testis', 'spleen', 'prostate', 'uterus', 'brain', 'liver', 'ovary'}

MAR03386 || Fatty acid oxidation
{'nerve', 'blood_vessel', 'small_intestine', 'esophagus', 'colon', 'cervix_uteri', 'pancreas', 'pituitary'}

MAR00638 || Glycerophospholipid metabolism
{'blood_vessel', 'small_intestine', 'muscle', 'skin', 'heart', 'esophagus', 'salivary_gland', 'blood'}

MAR01648 || Transport reactions
{'nerve', 'blood_vessel', 'small_intestine', 'esophagus', 'colon', 'cervix_uteri', 'pancreas', 'pituitary'}

MAR01645 || Transport reactions
{'nerve', 'blood_vessel', 'small_intestine', 'esophagus', 'colon', 'cervix_uteri', 'pancreas', 'pituitary'}

MAR04336 || Folate metabolism
{'blood_vessel', 'muscle', 'skin', 'heart', 'esophagus', 'salivary_gland', 'blood'}

MAR02594 || Carnitine shuttle (cytosolic)
{'nerve', 'small_intestine', 'breast', 'testis', 'prostate', 'uterus', 'brain', 'adrenal_gland', 'ovary'}

MAR00636 || Glycerophospholipid metabolism
{'blood_v

**Search for genes and metabolites involved in good MCSs**.
- Search and addition of all GPR rules in a dictionary called genes.
- Search and addition of the ID and the name of the reaction-forming metabolites for each good reaction.
- ID scanning of different gene databases (Entrez, Symbol and name) by accessing the corresponding API endpoint.

In [24]:
genes=dict()
for r in modeloBase.reactions:
    if len(r.genes)==0:
        genes[r.id]=set()
    if len(r.genes)==1:
        genes[r.id]=set([g.id for g in r.genes])
    if len(r.genes)>=2:
        exp=r.gene_reaction_rule
        if "and" in exp and not "or" in exp:
            genes[r.id]=set([g.id for g in r.genes])
        if "or" in exp and not "and" in exp:
            genes[r.id]=set([g.id for g in r.genes])
        if "and" in exp and "or" in exp:
            genes[r.id]=set([g.id for g in r.genes])

In [25]:
metabolites = dict()
names_met = dict()
for r in modeloBase.reactions:
    metabolite_list = [metabolite.id for metabolite in r.metabolites]
    met_names = [metabolite.name for metabolite in r.metabolites]
    metabolites[r.id] = metabolite_list
    names_met[r.id] = met_names

In [26]:
genes_interes = []
nombres_metabolitos_unicos = set() 
for r in set(buenasLinea).difference(buenasTodas):
    print(r, "Genes asociados", genes[r])
    genes_interes.append(genes[r])
    print("Metabolitos asociados", metabolites[r])
    print("Metabolitos names", names_met[r])
    nombres_metabolitos_unicos.update(names_met[r]) 
    print()
    print()

nombres_metabolitos_unicos_lista = list(nombres_metabolitos_unicos)

MAR07708 Genes asociados set()
Metabolitos asociados ['MAM02007c', 'MAM02007m']
Metabolitos names ['glyoxalate', 'glyoxalate']


MAR06464 Genes asociados {'ENSG00000186115', 'ENSG00000186526', 'ENSG00000171903', 'ENSG00000171954', 'ENSG00000186529'}
Metabolitos asociados ['MAM01935c', 'MAM02039c', 'MAM02555c', 'MAM02630c', 'MAM00360c', 'MAM02040c', 'MAM02554c']
Metabolitos names ['gamma-tocopherol', 'H+', 'NADPH', 'O2', '13-hydroxy-gamma-tocopherol', 'H2O', 'NADP+']


MAR20111 Genes asociados {'ENSG00000125246'}
Metabolitos asociados ['MAM01261m', 'MAM02007m', 'MAM02040m', 'MAM01597m', 'MAM02439m']
Metabolitos names ['acetyl-CoA', 'glyoxalate', 'H2O', 'CoA', 'malate']


MAR04922 Genes asociados set()
Metabolitos asociados ['MAM01596c', 'MAM01596m']
Metabolitos names ['CO2', 'CO2']


MAR08009 Genes asociados set()
Metabolitos asociados ['MAM01415c', 'MAM01415m']
Metabolitos names ['calcidiol', 'calcidiol']


MAR07702 Genes asociados {'ENSG00000137106'}
Metabolitos asociados ['MAM02007c'

In [27]:
for elemento in genes_interes[:]: 
    if isinstance(elemento, set) and not elemento:
        genes_interes.remove(elemento)
print(genes_interes)

[{'ENSG00000186115', 'ENSG00000186526', 'ENSG00000171903', 'ENSG00000171954', 'ENSG00000186529'}, {'ENSG00000125246'}, {'ENSG00000137106'}, {'ENSG00000114113', 'ENSG00000114115'}, {'ENSG00000135437'}, {'ENSG00000186115', 'ENSG00000186526', 'ENSG00000171903', 'ENSG00000171954', 'ENSG00000186529'}]


In [28]:
genes_a_buscar = [gene for conjunto in genes_interes for gene in conjunto]
mg = mygene.MyGeneInfo()
df = mg.querymany(genes_a_buscar, scopes='ensembl.gene', species=9606, as_dataframe=True)
df_sin_duplicados = df.drop_duplicates()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df_sin_duplicados.iloc[:, [2, 3, 4]]

5 input query terms found dup hits:	[('ENSG00000186115', 2), ('ENSG00000186526', 2), ('ENSG00000171903', 2), ('ENSG00000171954', 2), ('E


,entrezgene,name,symbol
query,,,
ENSG00000186115,8529,cytochrome P450 family 4 subfamily F member 2,CYP4F2
ENSG00000186526,11283,cytochrome P450 family 4 subfamily F member 8,CYP4F8
ENSG00000171903,57834,cytochrome P450 family 4 subfamily F member 11,CYP4F11
ENSG00000171954,126410,cytochrome P450 family 4 subfamily F member 22,CYP4F22
ENSG00000186529,4051,cytochrome P450 family 4 subfamily F member 3,CYP4F3
ENSG00000125246,171425,citramalyl-CoA lyase,CLYBL
ENSG00000137106,9380,glyoxylate and hydroxypyruvate reductase,GRHPR
ENSG00000114113,5948,retinol binding protein 2,RBP2
ENSG00000114115,5947,retinol binding protein 1,RBP1


In [29]:
symbol_dict={}
for g in genes_a_buscar:
    gene = mg.getgene(g, fields='symbol')
    if "symbol" in gene:
        symbol_dict[g]= gene["symbol"]
symbol_dict

{'ENSG00000186115': 'CYP4F2',
 'ENSG00000186526': 'CYP4F8',
 'ENSG00000171903': 'CYP4F11',
 'ENSG00000171954': 'CYP4F22',
 'ENSG00000186529': 'CYP4F3',
 'ENSG00000125246': 'CLYBL',
 'ENSG00000137106': 'GRHPR',
 'ENSG00000114113': 'RBP2',
 'ENSG00000114115': 'RBP1',
 'ENSG00000135437': 'RDH5'}

**Drug search for diseases, genes and coincidences**
- We access the chEMBL API endpoint and retrieve any molecule that has activity against our genes of interest that control the most promising reactions (MCSs).
- The idea is to see if we can propose a drug that attacks our therapeutic targets in a way that might be interesting to explore our results experimentally.

**Get approved drugs for MM**

In [30]:
drug_indication = new_client.drug_indication
molecules = new_client.molecule

MM_ind = drug_indication.filter(efo_term__icontains="MULTIPLE MYELOMA")
MM_mols = molecules.filter(
    molecule_chembl_id__in=[x['molecule_chembl_id'] for x in MM_ind])

len(MM_mols)

436

**Get approved drugs for cancer**

In [31]:
drug_indication = new_client.drug_indication
molecules = new_client.molecule

cancer_ind = drug_indication.filter(efo_term__icontains="CANCER")
cancer_mols = molecules.filter(
    molecule_chembl_id__in=[x['molecule_chembl_id'] for x in cancer_ind])

len(cancer_mols)

1734

In [32]:
df_MM_mols = pd.DataFrame(MM_mols)
df_MM_pref_name = df_MM_mols[['pref_name']]
df_cancer_mols = pd.DataFrame(cancer_mols)
df_cancer_pref_name = df_cancer_mols[['pref_name']]
df_coincidencias_diseases = pd.merge(df_MM_pref_name, df_cancer_pref_name, on="pref_name")
df_outer = pd.merge(df_MM_pref_name, df_cancer_pref_name, on="pref_name", how="outer", indicator=True)
df_no_coincidencias_cancer = df_outer[df_outer["_merge"] == "right_only"]
df_no_coincidencias_final_cancer = df_no_coincidencias_cancer[df_cancer_pref_name.columns]

In [33]:
def buscar_drugs(gene_symbol):
    target = new_client.target
    gene_name = gene_symbol
    res = target.filter(target_synonym__icontains=gene_name).only(['organism'["organism" == "Homo sapiens"], 'target_chembl_id'])
    chembl_id = None
    for i in res:
        chembl_id=i["target_chembl_id"]
        break
     
    activity = new_client.activity
    res2 = activity.filter(target_chembl_id=chembl_id, assay_type='B').only(["target_organism"["target_organism" == "Homo sapiens"],
                                                                                'target_chembl_id', 
                                                                                'target_pref_name', 
                                                                                'molecule_pref_name'])

    if len(res2) > 0:
        res2 = pd.DataFrame(res2)
        res2["gen_symbol"] = gene_name
        res2 = res2[res2["molecule_pref_name"].notna()]
        return res2

In [34]:
resultados_drogas=pd.DataFrame({})
for g in symbol_dict:
    gen= symbol_dict[g]
    drogas=buscar_drugs(gen)
    resultados_drogas=pd.concat([resultados_drogas, drogas], ignore_index=True)
df_buenos_drugs = resultados_drogas.drop_duplicates()
df_buenos_drugs

,molecule_pref_name,target_chembl_id,target_pref_name,gen_symbol
0,ENOXACIN,CHEMBL2098,TAR RNA binding protein 2,RBP2


In [35]:
pd.DataFrame(df_buenos_drugs["gen_symbol"].value_counts())

,gen_symbol
RBP2,1


In [36]:
df_coincidencias_MM = pd.merge(df_MM_pref_name, df_buenos_drugs, left_on="pref_name", right_on="molecule_pref_name")
print("\nDataFrame with matches between approved MM drugs and drugs associated with our genes of interest:")
df_coincidencias_MM
df_coincidencias_cancer = pd.merge(df_no_coincidencias_final_cancer, df_buenos_drugs, left_on="pref_name", right_on="molecule_pref_name")
print("\nDataFrame with matches between approved cáncer drugs and drugs associated with our genes of interest:")
df_coincidencias_cancer


DataFrame with matches between approved MM drugs and drugs associated with our genes of interest:

DataFrame with matches between approved cáncer drugs and drugs associated with our genes of interest:


,pref_name,molecule_pref_name,target_chembl_id,target_pref_name,gen_symbol


**The next step will be to analyse how the reactions that emerge as new MCS affect the rest of the tissues and lines**

- To do this, we are going to load all the MCS of MM lines, since we only had the MCS of healthy tissues loaded.
- Next, we are going to generate two dictionaries where we will store the number of MM models affected and in another one the name of those affected.

**Target studies (MCSs that affect MM lines)**
- modelsMMAffected[r.id] : list of MM lines affected by some MCSs
- numberMMAffectedModels[r.id]: list with the number of toxicities MM lines
- Level1,..., Level5: level of scope MM
- generic: MCS of the base model
- someModelMM: MCSs which block some line
- specificMM: MCSs which block some line and are not generic

In [37]:
for nombre in nombresMM:
    nombre=nombre[:len(nombre)-7]
    with open("/home/alumno15/MCS/MM/"+nombre+"MCS.dat","rb") as f:
        MCS=pickle.load(f)
    todos=set()
    for task in range(57):
        todos.update(MCS[task][0])
    ByTask=[set()]
    for task in range(57):
        ByTask.append(MCS[task][0])
    MCSByTask[nombre]=ByTask
    MCSAllTasks[nombre]=todos

In [38]:
algunModeloMM=set()
for i in range(31,49):
    especificasTask=MCSAllTasks[nombreModelo[i]]
    algunModeloMM.update(especificasTask)
print("Number of MCSs in some MM line model:",len(algunModeloMM))

Number of MCSs in some MM line model: 972


In [39]:
especificasMM=algunModeloMM.difference(genericas)
print("Number of MCSs specific to MM line models:",len(especificasMM))

Number of MCSs specific to MM line models: 531


In [40]:
numeroModelosMMAfectados={}
modelosMMAfectados={}
for r in especificasMM:
    afectados=set()
    for i in range(31,49):
        if r in MCSAllTasks[nombreModelo[i]]:
            afectados.add(nombreModelo[i])
    numeroModelosMMAfectados[r]=len(afectados)
    modelosMMAfectados[r]=afectados
    
# Ampliamos modelosAfectados y numeroModelosAfectados para incluir estas
for r in especificasMM:
    if not r in modelosAfectados:
        numeroModelosAfectados[r]=0
        modelosAfectados[r]=set()

**Analysis of subsystem reactions and how they affect models other than the one considered for the MMIS line**

In [41]:
retinolNuevas=["MAR06662","MAR06645","MAR08710"]

In [42]:
for r in retinolNuevas:
    print(r,modelosAfectados[r])

MAR06662 set()
MAR06645 {'spleen', 'thyroid', 'esophagus', 'brain'}
MAR08710 {'spleen', 'thyroid', 'esophagus', 'brain'}


In [43]:
for r in retinolNuevas:
    print(r,modelosMMAfectados[r])

MAR06662 {'ACH_000588', 'ACH_000763', 'ACH_000829'}
MAR06645 {'ACH_000363', 'ACH_000576', 'ACH_000714', 'ACH_000512', 'ACH_000838', 'ACH_000889', 'ACH_000183', 'ACH_000763'}
MAR08710 {'ACH_000363', 'ACH_000576', 'ACH_000714', 'ACH_000512', 'ACH_000838', 'ACH_000889', 'ACH_000183', 'ACH_000763'}


In [44]:
transporteNuevas=set(["MAR07708","MAR08009","MAR02299","MAR04922"])
for r in transporteNuevas:
    print(r)
    reaccion=modeloBase.reactions.get_by_id(r)
    for m in reaccion.metabolites:
        print(m.name,m.id)
    print()

MAR07708
glyoxalate MAM02007c
glyoxalate MAM02007m

MAR08009
calcidiol MAM01415c
calcidiol MAM01415m

MAR04922
CO2 MAM01596c
CO2 MAM01596m

MAR02299
tyrosine MAM03101c
tyrosine MAM03101m



In [45]:
for r in transporteNuevas:
    print(r,modelosAfectados[r])

MAR07708 set()
MAR08009 set()
MAR04922 set()
MAR02299 {'blood_vessel', 'pancreas', 'heart'}


In [46]:
for r in transporteNuevas:
    print(r,numeroModelosMMAfectados[r],numeroModelosAfectados[r],modeloBase.reactions.get_by_id(r).gene_reaction_rule)

MAR07708 5 0 
MAR08009 8 0 
MAR04922 7 0 
MAR02299 6 3 


In [47]:
r="MAR20111"
print(r,numeroModelosMMAfectados[r],numeroModelosAfectados[r],modeloBase.reactions.get_by_id(r).gene_reaction_rule)
print(modelosMMAfectados[r])

MAR20111 5 0 ENSG00000125246
{'ACH_000363', 'ACH_000426', 'ACH_000829', 'ACH_000889', 'ACH_000763'}


In [48]:
r="MAR07708"
print(r,numeroModelosMMAfectados[r],numeroModelosAfectados[r],modeloBase.reactions.get_by_id(r).gene_reaction_rule)
print(modelosMMAfectados[r])

MAR07708 5 0 
{'ACH_000363', 'ACH_000426', 'ACH_000829', 'ACH_000889', 'ACH_000763'}


In [49]:
r="MAR06464"
print(r,numeroModelosMMAfectados[r],numeroModelosAfectados[r],modeloBase.reactions.get_by_id(r).gene_reaction_rule)
print(modelosMMAfectados[r])
print(modelosAfectados[r])

MAR06464 5 4 ENSG00000171903 or ENSG00000171954 or ENSG00000186115 or ENSG00000186526 or ENSG00000186529
{'ACH_000363', 'ACH_000821', 'ACH_000183', 'ACH_000763', 'ACH_000653'}
{'esophagus', 'colon', 'heart', 'ovary'}


In [50]:
r="MAR06466"
print(r,numeroModelosMMAfectados[r],numeroModelosAfectados[r],modeloBase.reactions.get_by_id(r).gene_reaction_rule)
print(modelosMMAfectados[r])
print(modelosAfectados[r])

MAR06466 5 4 ENSG00000171903 or ENSG00000171954 or ENSG00000186115 or ENSG00000186526 or ENSG00000186529
{'ACH_000363', 'ACH_000821', 'ACH_000183', 'ACH_000763', 'ACH_000653'}
{'esophagus', 'colon', 'heart', 'ovary'}


In [51]:
r="MAR07702"
print(r,numeroModelosMMAfectados[r],numeroModelosAfectados[r],modeloBase.reactions.get_by_id(r).gene_reaction_rule)
print(modelosMMAfectados[r])
print(modelosAfectados[r])

MAR07702 5 0 ENSG00000137106
{'ACH_000363', 'ACH_000426', 'ACH_000829', 'ACH_000889', 'ACH_000763'}
set()
